In [1]:
pip install --upgrade-strategy eager install optimum[onnxruntime]


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install git+https://github.com/huggingface/transformers.git@main

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /private/var/folders/hr/pk_thz1s2w1b7rh7_xc8vxrh0000gn/T/pip-req-build-fy2vjzxe
  Running command git clone -q https://github.com/huggingface/transformers.git /private/var/folders/hr/pk_thz1s2w1b7rh7_xc8vxrh0000gn/T/pip-req-build-fy2vjzxe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8551656 sha256=857edd367d7ed88b6cee6c328476bbca4bc6ab98542fc5af64dc107706a47624
  Stored in directory: /private/var/folders/hr/pk_thz1s2w1b7rh7_xc8vxrh0000gn/T/pip-ephem-wheel-cache-eg8dyil6/wheels/0c/f1/cf/0c84f8631406672e9adab41401961ab0d771b0b9c6f7195624
Successfully built transformers
ERROR: spacy-transformers 1.1.9 has requirement transformers<4.26.0,>=3.4.0, but you'll have transformers 4.39.0.dev0 which is incompatible.
ERROR: fastt5 0.1.4 has requiremen

In [2]:
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForTokenClassification

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
quantized_model = ORTModelForTokenClassification.from_pretrained("/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/model-quantized")
tokenizerdisq = AutoTokenizer.from_pretrained("/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/model-quantized")
onnx_ner = pipeline("token-classification", model=quantized_model, tokenizer=tokenizerdisq, aggregation_strategy="first")

In [13]:
def sanitise_data(text: str, entities: list) -> dict:
    return_entities=[]
    for entity in entities:
        santitise_entity={}
        santitise_entity['start'] = entity['start']
        santitise_entity['end'] = entity['end']
        santitise_entity['label'] = entity['entity_group']
        return_entities.append(santitise_entity)
    return {'text':text, 'ents':return_entities, 'title': None}

In [6]:
text='In the bustling city of New York, nestled amidst the towering skyscrapers, lived two childhood friends, Emma and Jack. \
    They grew up in the vibrant neighborhood of Brooklyn, their bond forged through countless adventures and shared dreams. \
    Despite their different backgrounds, they were inseparable, their friendship standing the test of time. As they entered adulthood,\
    Emma pursued her passion for art, while Jack found success in the world of finance. Despite their divergent paths, they remained close, \
    supporting each other through lifes ups and downs. However, their lives took an unexpected turn when the art gallery, "Canvas Creations," \
    faced financial difficulties. Struggling to keep her business afloat, Emma found herself on the brink of despair. Determined to help his \
    friend, Jack proposed a daring plan. He was a member of a secret organization known as "The Brotherhood," a clandestine group dedicated to \
    aiding those in need. With their vast resources and network of contacts, The Brotherhood had the power to solve Emmas problems discreetly. \
    Despite Emmas initial reluctance, she eventually agreed, desperate to save her gallery from ruin. Under the cover of darkness, Jack \
    arranged a clandestine meeting with The Brotherhoods enigmatic leader, known only as "The Guardian." In a dimly lit alleyway, Jack \
    explained Emmas predicament, pleading for The Brotherhoods assistance. Impressed by Jacks loyalty and determination, The Guardian agreed \
    to help, on one condition: Emma and Jack would owe The Brotherhood a favor, to be redeemed at a later date. With The Brotherhoods support,\
    Emmas gallery was saved from bankruptcy. Grateful for their assistance, Emma and Jack resumed their lives, hoping to put the ordeal \
    behind them. However, their newfound peace was short-lived when The Brotherhood called in their favor. Emma and Jack found themselves \
    embroiled in a dangerous mission, tasked with recovering a valuable artifact stolen from The Brotherhoods archives. With their lives on \
    the line, they embarked on a perilous journey, spanning continents and facing formidable adversaries. Along the way, they uncovered \
    secrets that shook the foundations of The Brotherhood, challenging their loyalty and trust. Despite the odds stacked against them, \
    Emma and Jack persevered, their friendship and determination guiding them through adversity. With their unwavering resolve, they \
    succeeded in retrieving the stolen artifact, earning The Brotherhoods gratitude and respect. However, their victory came at a cost:\
    they were forever bound to The Brotherhood, their lives intertwined with the clandestine organizations machinations. As they returned \
    to New York, Emma and Jack reflected on their harrowing ordeal, knowing that their lives would never be the same. Despite the dangers\
     that lay ahead, they faced the future together, united by their friendship and the bonds they had forged. For in the shadows of the \
    city that never sleeps, secrets lurked, and alliances shifted – but Emma and Jack knew that as long as they had each other, they could \
    overcome any challenge that came their way.'

In [17]:
import time
start_time=time.time()
pred = onnx_ner(text)
pred = sanitise_data(text, pred)
end_time=time.time()
print (f"The time to return results for 504 words of text is: {(end_time-start_time):.2f}")

The time to return results for 504 words of text is: 0.80


In [6]:
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [7]:
# model_id="/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model"
model_id='/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model/model-quantized'
tokenizer_id='/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:'

In [8]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
model = ORTModelForTokenClassification.from_pretrained(model_id)
# model = ORTModelForTokenClassification.from_pretrained("jammmmmm/pii", export=True)
tokenizer.model_input_names = ['input_ids', 'attention_mask']

In [47]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(model_id)
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model
model_quantized_path = quantizer.quantize(
    save_dir=model_id + "/model-quantized.onnx",
    quantization_config=qconfig,
)

RuntimeError: Found too many ONNX model files in /Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model. ORTQuantizer does not support multi-file quantization. Please create separate ORTQuantizer instances for each model/file, by passing the argument `file_name` to ORTQuantizer.from_pretrained().

In [55]:
quantized_model=ORTModelForTokenClassification.from_pretrained("/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model/model-quantized")

In [54]:
import os
dir = "/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model/model-quantized"
os.path.isdir(dir)

True

In [9]:
onnx_ner = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="first")

In [10]:
text = "My name is John. "
pred = onnx_ner(text)
print(pred)

[{'entity_group': 'FIRSTNAME', 'score': 0.80968964, 'word': ' John.', 'start': 10, 'end': 16}]


/usr/local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [15]:
class NEROnnxModel():
    def __call__(self, text: str) -> str:
        # load quantized model and tokenizers
        # quantized_model = ORTModelForTokenClassification.from_pretrained("/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/model-quantized")
        # tokenizerdisq = AutoTokenizer.from_pretrained("/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/model-quantized")

        model_id='/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/ner/model/model-quantized'
        tokenizer_id='/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:'

        tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
        model = ORTModelForTokenClassification.from_pretrained(model_id)
        tokenizer.model_input_names = ['input_ids', 'attention_mask']       
        #create transformers pipeline
        # onnx_ner = pipeline("token-classification", model=quantized_model, tokenizer=tokenizerdisq, aggregation_strategy="first")
        onnx_ner = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="first")

        #run the pipeline and return the results
        pred = onnx_ner(text)
        print(pred)
        pred = self.sanitise_data(text, pred)
        print(pred)
        return pred
    
    def sanitise_data(self, text: str, entities: list) -> dict:
        return_entities=[]
        for entity in entities:
            santitise_entity={}
            santitise_entity['start'] = entity['start']
            santitise_entity['end'] = entity['end']
            santitise_entity['label'] = entity['entity_group']
            return_entities.append(santitise_entity)
        return {'text':text, 'ents':return_entities, 'title': None}
    
if __name__ == '__main__':
    text = "Taylor Swift lives in London."
    pipe = NEROnnxModel()
    results = pipe(text)
    print(results)

[{'entity_group': 'CITY', 'score': 0.8359974, 'word': ' London.', 'start': 21, 'end': 29}]
{'text': 'Taylor Swift lives in London.', 'ents': [{'start': 21, 'end': 29, 'label': 'CITY'}], 'title': None}
{'text': 'Taylor Swift lives in London.', 'ents': [{'start': 21, 'end': 29, 'label': 'CITY'}], 'title': None}


In [104]:
import numpy as np
from onnxruntime import InferenceSession, SessionOptions, GraphOptimizationLevel
from transformers import AutoTokenizer
from typing import List, Tuple
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument

def create_model_for_provider(model_path, provider='CPUExecutionProvider'):
    '''
    Create CPU inference session for ONNX runtime to boost performance
    :param model_path: path to *.onnx model
    :param provider: CPU/CUDA
    :return: onnx runtime session
    '''
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    print(session)
    return session

class NEROnnxModel():
    """Build NER onnx model and aggregate results into data to be rendered"""
    def __init__(self, quant=True) -> None:
        # if quant:
        model_path = "/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/quant/model.onnx"
        # else: 
        #     model_path = "/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/ner/model/model.onnx"
        self.model = create_model_for_provider(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained('/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:')
        self.id2label= {
    "0": "B-PREFIX",
    "1": "I-PREFIX",
    "2": "B-FIRSTNAME",
    "3": "I-FIRSTNAME",
    "4": "B-MIDDLENAME",
    "5": "B-LASTNAME",
    "6": "I-LASTNAME",
    "7": "O",
    "8": "B-JOBDESCRIPTOR",
    "9": "B-JOBTITLE",
    "10": "I-JOBTITLE",
    "11": "B-COMPANY_NAME",
    "12": "I-COMPANY_NAME",
    "13": "B-JOBAREA",
    "14": "B-EMAIL",
    "15": "I-EMAIL",
    "16": "B-TIME",
    "17": "I-TIME",
    "18": "B-DATE",
    "19": "I-DATE",
    "20": "B-URL",
    "21": "I-URL",
    "22": "B-BITCOINADDRESS",
    "23": "I-BITCOINADDRESS",
    "24": "B-ETHEREUMADDRESS",
    "25": "I-ETHEREUMADDRESS",
    "26": "B-ACCOUNTNAME",
    "27": "I-ACCOUNTNAME",
    "28": "B-IBAN",
    "29": "I-IBAN",
    "30": "B-ACCOUNTNUMBER",
    "31": "I-ACCOUNTNUMBER",
    "32": "B-BIC",
    "33": "I-BIC",
    "34": "B-IPV4",
    "35": "I-IPV4",
    "36": "B-STREETADDRESS",
    "37": "I-STREETADDRESS",
    "38": "B-CITY",
    "39": "I-CITY",
    "40": "B-ZIPCODE",
    "41": "I-ZIPCODE",
    "42": "B-USERNAME",
    "43": "I-USERNAME",
    "44": "B-IPV6",
    "45": "I-IPV6",
    "46": "B-CREDITCARDNUMBER",
    "47": "I-CREDITCARDNUMBER",
    "48": "B-VEHICLEVIN",
    "49": "I-VEHICLEVIN",
    "50": "B-SUFFIX",
    "51": "I-SUFFIX",
    "52": "B-AMOUNT",
    "53": "I-AMOUNT",
    "54": "B-CURRENCY",
    "55": "I-CURRENCY",
    "56": "B-PASSWORD",
    "57": "I-PASSWORD",
    "58": "B-JOBTYPE",
    "59": "B-STATE",
    "60": "B-BUILDINGNUMBER",
    "61": "I-BUILDINGNUMBER",
    "62": "B-VEHICLEVRM",
    "63": "I-VEHICLEVRM",
    "64": "B-PHONEIMEI",
    "65": "I-PHONEIMEI",
    "66": "I-JOBAREA",
    "67": "I-STATE",
    "68": "B-COUNTY",
    "69": "B-CURRENCYNAME",
    "70": "I-CURRENCYNAME",
    "71": "B-CURRENCYSYMBOL",
    "72": "B-MASKEDNUMBER",
    "73": "I-MASKEDNUMBER",
    "74": "B-PHONE_NUMBER",
    "75": "I-PHONE_NUMBER",
    "76": "B-SECONDARYADDRESS",
    "77": "I-SECONDARYADDRESS",
    "78": "B-SSN",
    "79": "I-SSN",
    "80": "B-CURRENCYCODE",
    "81": "B-LITECOINADDRESS",
    "82": "I-LITECOINADDRESS",
    "83": "B-MAC",
    "84": "I-MAC",
    "85": "B-CREDITCARDISSUER",
    "86": "I-CREDITCARDISSUER",
    "87": "B-CREDITCARDCVV",
    "88": "I-CREDITCARDCVV",
    "89": "B-USERAGENT",
    "90": "I-USERAGENT",
    "91": "B-IP",
    "92": "I-IP",
    "93": "B-SEX",
    "94": "B-STREET",
    "95": "I-STREET",
    "96": "B-PIN",
    "97": "I-PIN",
    "98": "I-JOBTYPE",
    "99": "I-MIDDLENAME",
    "100": "I-CURRENCYCODE",
    "101": "I-CURRENCYSYMBOL",
    "102": "B-FULLNAME",
    "103": "I-FULLNAME",
    "104": "B-NAME",
    "105": "I-NAME",
    "106": "B-GENDER",
    "107": "B-NUMBER",
    "108": "I-NUMBER",
    "109": "I-GENDER",
    "110": "B-NEARBYGPSCOORDINATE",
    "111": "I-NEARBYGPSCOORDINATE",
    "112": "B-DISPLAYNAME",
    "113": "I-DISPLAYNAME",
    "114": "B-SEXTYPE",
    "115": "B-ORDINALDIRECTION"
  }

    def __call__(self, sentence: str) -> str:
        # get model inputs and other required arrays
        model_inputs = self.tokenizer(sentence, return_tensors="np",
            return_special_tokens_mask=True, return_offsets_mapping=True)
        special_tokens_mask = model_inputs.pop("special_tokens_mask")[0]
        offset_mapping = model_inputs.pop("offset_mapping")[0]
        logits = None
        print(model_inputs)
        try:
            logits = self.model.run(None, dict(model_inputs))[0]
        except InvalidArgument as e:
            if 'token_type_ids' in str(e):
                model_inputs.pop('token_type_ids', None)
        # pass to model
        logits = self.model.run(None, dict(model_inputs))[0]
        print(np.argmax(logits, axis=2))
        predictions = np.argmax(logits, axis=2)[0]
        input_ids = model_inputs["input_ids"][0]
        model_outputs = {"sentence": sentence, "input_ids": input_ids, "predictions": predictions, 
            "offset_mapping": offset_mapping, "special_tokens_mask": special_tokens_mask}
        print(model_outputs)
        # aggregate entitity information
        pre_entities = self.gather_pre_entities(**model_outputs)
        entities = self.aggregate_words(pre_entities)
        entities = self.group_entities(entities)
        results = self.build_final_output(sentence, entities)
        return results

    def gather_pre_entities(self, sentence: str, input_ids: np.ndarray, predictions: List[int],
            offset_mapping: List[Tuple[int, int]], special_tokens_mask: np.ndarray) -> List[dict]:
        """Fuse various numpy arrays into dicts with all the information needed for aggregation"""
        pre_entities = []
        for idx, pred in enumerate(predictions):
            # Filter special_tokens, they should only occur at the sentence boundaries 
            if special_tokens_mask[idx]:
                continue
            word = self.tokenizer.convert_ids_to_tokens(int(input_ids[idx]))
            start_ind, end_ind = offset_mapping[idx]
            word_ref = sentence[start_ind:end_ind]
            is_subword = len(word) != len(word_ref)
            if int(input_ids[idx]) == self.tokenizer.unk_token_id:
                word = word_ref
                is_subword = False
            pre_entity = {"word": word, "entity": self.id2label[str(pred)], "start": start_ind,
                "end": end_ind, "index": idx, "is_subword": is_subword}
            pre_entities.append(pre_entity)
        return pre_entities

    def aggregate_word(self, entities: List[dict]) -> dict:
        """Aggregate sub-words together and make sure entities align"""
        word = self.tokenizer.convert_tokens_to_string([entity["word"] for entity in entities])
        entity = entities[0]["entity"]
        new_entity = {"word": word, "entity": entity, "start": entities[0]["start"],
            "end": entities[-1]["end"]}
        return new_entity

    def aggregate_words(self, entities: List[dict]) -> List[dict]:
        """Override tokens from a given word that disagree to force agreement on word boundaries"""
        word_entities = []
        word_group = None
        for entity in entities:
            if word_group is None:
                word_group = [entity]
            elif entity["is_subword"]:
                word_group.append(entity)
            else:
                word_entities.append(self.aggregate_word(word_group))
                word_group = [entity]
        # add last item
        word_entities.append(self.aggregate_word(word_group))
        return word_entities

    def group_sub_entities(self, entities: List[dict]) -> dict:
        """Group together the adjacent tokens with the same entity predicted"""
        # Get the first entity in the entity group
        entity = entities[0]["entity"].split("-")[-1]
        tokens = [entity["word"] for entity in entities]
        entity_group = {"entity_group": entity, "word": self.tokenizer.convert_tokens_to_string(tokens),
            "start": entities[0]["start"], "end": entities[-1]["end"]}
        return entity_group

    def get_tag(self, entity_name: str) -> Tuple[str, str]:
        if entity_name.startswith("B-"):
            bi = "B"
            tag = entity_name[2:]
        elif entity_name.startswith("I-"):
            bi = "I"
            tag = entity_name[2:]
        else:
            # if not in B-, I- format default to I- for continuation
            bi = "I"
            tag = entity_name
        return bi, tag

    def group_entities(self, entities: List[dict]) -> List[dict]:
        """Find and group together the adjacent tokens with the same entity predicted"""
        entity_groups = []
        entity_group_disagg = []
        for entity in entities:
            if not entity_group_disagg:
                entity_group_disagg.append(entity)
                continue
            # if the current entity is similar and adjacent to the previous entity, 
            # append it to the disaggregated entity group
            bi, tag = self.get_tag(entity["entity"])
            last_bi, last_tag = self.get_tag(entity_group_disagg[-1]["entity"])
            if tag == last_tag and bi != "B":
                # modify subword type to be previous_type
                entity_group_disagg.append(entity)
            else:
                # if the current entity is different from the previous entity
                # aggregate the disaggregated entity group
                entity_groups.append(self.group_sub_entities(entity_group_disagg))
                entity_group_disagg = [entity]
        if entity_group_disagg:
            # it's the last entity, add it to the entity groups
            entity_groups.append(self.group_sub_entities(entity_group_disagg))
        return entity_groups

    def build_final_output(self, sentence: str, entity_groups: List[dict]) -> List[dict]:
        entities = []
        for entity_group in entity_groups:
            if entity_group['entity_group'] == 'O':
                continue
            else:
                entry = {}
                entry['start'] = entity_group['start']
                entry['end'] = entity_group['end']
                entry['label'] = entity_group['entity_group']
                entities.append(entry)
        render_data = {'text': sentence, 'ents': entities, 'title': None}
        return render_data
    
if __name__ == '__main__':
    # model_checkpoint = "ner/model/model.onnx"
    sentence = "Jack Sparrow lives in New York."
    # sentence = "Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879. Six weeks later the family moved to Munich, where he later on began his schooling at the Luitpold Gymnasium. Later, they moved to Italy and Albert continued his education at Aarau, Switzerland and in 1896 he entered the Swiss Federal Polytechnic School in Zurich to be trained as a teacher in physics and mathematics."
    pipe = NEROnnxModel()
    results = pipe(sentence)
    print(results)

{'input_ids': array([[    1,  2722, 36671,  1074,    11,   188,   469,     4,     2]]), 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[[  7   7 103   7   7  59  67   7   7]]
{'sentence': 'Jack Sparrow lives in New York.', 'input_ids': array([    1,  2722, 36671,  1074,    11,   188,   469,     4,     2]), 'predictions': array([  7,   7, 103,   7,   7,  59,  67,   7,   7]), 'offset_mapping': array([[ 0,  0],
       [ 0,  4],
       [ 4, 12],
       [12, 18],
       [18, 21],
       [21, 25],
       [25, 30],
       [30, 31],
       [ 0,  0]]), 'special_tokens_mask': array([1, 0, 0, 0, 0, 0, 0, 0, 1])}
{'text': 'Jack Sparrow lives in New York.', 'ents': [{'start': 4, 'end': 12, 'label': 'FULLNAME'}, {'start': 21, 'end': 30, 'label': 'STATE'}], 'title': None}


In [108]:
import numpy as np
from onnxruntime import InferenceSession, SessionOptions, GraphOptimizationLevel
from transformers import AutoTokenizer
from typing import List, Tuple
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument

def create_model_for_provider(model_path, provider='CPUExecutionProvider'):
    '''
    Create CPU inference session for ONNX runtime to boost performance
    :param model_path: path to *.onnx model
    :param provider: CPU/CUDA
    :return: onnx runtime session
    '''
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    return session

class NEROnnxModel():
    """Build NER onnx model and aggregate results into data to be rendered"""
    def __init__(self, quant=True) -> None:
        # if quant:
        model_path = "/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:/quant/model-quant1.onnx"
        # else: 
        #     model_path = "/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/ner/model/model.onnx"
        self.model = create_model_for_provider(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained('/Users/salonisalian/Desktop/University Files/Year 3/third_year_project/pii_onnx:')
        self.id2label= {
    "0": "B-PREFIX",
    "1": "I-PREFIX",
    "2": "B-FIRSTNAME",
    "3": "I-FIRSTNAME",
    "4": "B-MIDDLENAME",
    "5": "B-LASTNAME",
    "6": "I-LASTNAME",
    "7": "O",
    "8": "B-JOBDESCRIPTOR",
    "9": "B-JOBTITLE",
    "10": "I-JOBTITLE",
    "11": "B-COMPANY_NAME",
    "12": "I-COMPANY_NAME",
    "13": "B-JOBAREA",
    "14": "B-EMAIL",
    "15": "I-EMAIL",
    "16": "B-TIME",
    "17": "I-TIME",
    "18": "B-DATE",
    "19": "I-DATE",
    "20": "B-URL",
    "21": "I-URL",
    "22": "B-BITCOINADDRESS",
    "23": "I-BITCOINADDRESS",
    "24": "B-ETHEREUMADDRESS",
    "25": "I-ETHEREUMADDRESS",
    "26": "B-ACCOUNTNAME",
    "27": "I-ACCOUNTNAME",
    "28": "B-IBAN",
    "29": "I-IBAN",
    "30": "B-ACCOUNTNUMBER",
    "31": "I-ACCOUNTNUMBER",
    "32": "B-BIC",
    "33": "I-BIC",
    "34": "B-IPV4",
    "35": "I-IPV4",
    "36": "B-STREETADDRESS",
    "37": "I-STREETADDRESS",
    "38": "B-CITY",
    "39": "I-CITY",
    "40": "B-ZIPCODE",
    "41": "I-ZIPCODE",
    "42": "B-USERNAME",
    "43": "I-USERNAME",
    "44": "B-IPV6",
    "45": "I-IPV6",
    "46": "B-CREDITCARDNUMBER",
    "47": "I-CREDITCARDNUMBER",
    "48": "B-VEHICLEVIN",
    "49": "I-VEHICLEVIN",
    "50": "B-SUFFIX",
    "51": "I-SUFFIX",
    "52": "B-AMOUNT",
    "53": "I-AMOUNT",
    "54": "B-CURRENCY",
    "55": "I-CURRENCY",
    "56": "B-PASSWORD",
    "57": "I-PASSWORD",
    "58": "B-JOBTYPE",
    "59": "B-STATE",
    "60": "B-BUILDINGNUMBER",
    "61": "I-BUILDINGNUMBER",
    "62": "B-VEHICLEVRM",
    "63": "I-VEHICLEVRM",
    "64": "B-PHONEIMEI",
    "65": "I-PHONEIMEI",
    "66": "I-JOBAREA",
    "67": "I-STATE",
    "68": "B-COUNTY",
    "69": "B-CURRENCYNAME",
    "70": "I-CURRENCYNAME",
    "71": "B-CURRENCYSYMBOL",
    "72": "B-MASKEDNUMBER",
    "73": "I-MASKEDNUMBER",
    "74": "B-PHONE_NUMBER",
    "75": "I-PHONE_NUMBER",
    "76": "B-SECONDARYADDRESS",
    "77": "I-SECONDARYADDRESS",
    "78": "B-SSN",
    "79": "I-SSN",
    "80": "B-CURRENCYCODE",
    "81": "B-LITECOINADDRESS",
    "82": "I-LITECOINADDRESS",
    "83": "B-MAC",
    "84": "I-MAC",
    "85": "B-CREDITCARDISSUER",
    "86": "I-CREDITCARDISSUER",
    "87": "B-CREDITCARDCVV",
    "88": "I-CREDITCARDCVV",
    "89": "B-USERAGENT",
    "90": "I-USERAGENT",
    "91": "B-IP",
    "92": "I-IP",
    "93": "B-SEX",
    "94": "B-STREET",
    "95": "I-STREET",
    "96": "B-PIN",
    "97": "I-PIN",
    "98": "I-JOBTYPE",
    "99": "I-MIDDLENAME",
    "100": "I-CURRENCYCODE",
    "101": "I-CURRENCYSYMBOL",
    "102": "B-FULLNAME",
    "103": "I-FULLNAME",
    "104": "B-NAME",
    "105": "I-NAME",
    "106": "B-GENDER",
    "107": "B-NUMBER",
    "108": "I-NUMBER",
    "109": "I-GENDER",
    "110": "B-NEARBYGPSCOORDINATE",
    "111": "I-NEARBYGPSCOORDINATE",
    "112": "B-DISPLAYNAME",
    "113": "I-DISPLAYNAME",
    "114": "B-SEXTYPE",
    "115": "B-ORDINALDIRECTION"
  }

    def __call__(self, sentence: str) -> str:
        # get model inputs and other required arrays
        model_inputs = self.tokenizer(sentence, return_tensors="np",
            return_special_tokens_mask=True, return_offsets_mapping=True)
        special_tokens_mask = model_inputs.pop("special_tokens_mask")[0]
        offset_mapping = model_inputs.pop("offset_mapping")[0]
        logits = None
        print(model_inputs)
        try:
            logits = self.model.run(None, dict(model_inputs))[0]
        except InvalidArgument as e:
            if 'token_type_ids' in str(e):
                model_inputs.pop('token_type_ids', None)
        # pass to model
        logits = self.model.run(None, dict(model_inputs))[0]
        print(np.argmax(logits, axis=2))
        predictions = np.argmax(logits, axis=2)[0]
        input_ids = model_inputs["input_ids"][0]
        model_outputs = {"sentence": sentence, "input_ids": input_ids, "predictions": predictions, 
            "offset_mapping": offset_mapping, "special_tokens_mask": special_tokens_mask}
        print(model_outputs)
        # aggregate entitity information
        pre_entities = self.gather_pre_entities(**model_outputs)
        entities = self.aggregate_words(pre_entities)
        entities = self.group_entities(entities)
        results = self.build_final_output(sentence, entities)
        return results

    def gather_pre_entities(self, sentence: str, input_ids: np.ndarray, predictions: List[int],
            offset_mapping: List[Tuple[int, int]], special_tokens_mask: np.ndarray) -> List[dict]:
        """Fuse various numpy arrays into dicts with all the information needed for aggregation"""
        pre_entities = []
        for idx, pred in enumerate(predictions):
            # Filter special_tokens, they should only occur at the sentence boundaries 
            if special_tokens_mask[idx]:
                continue
            word = self.tokenizer.convert_ids_to_tokens(int(input_ids[idx]))
            start_ind, end_ind = offset_mapping[idx]
            word_ref = sentence[start_ind:end_ind]
            is_subword = len(word) != len(word_ref)
            if int(input_ids[idx]) == self.tokenizer.unk_token_id:
                word = word_ref
                is_subword = False
            pre_entity = {"word": word, "entity": self.id2label[str(pred)], "start": start_ind,
                "end": end_ind, "index": idx, "is_subword": is_subword}
            pre_entities.append(pre_entity)
        return pre_entities

    def aggregate_word(self, entities: List[dict]) -> dict:
        """Aggregate sub-words together and make sure entities align"""
        word = self.tokenizer.convert_tokens_to_string([entity["word"] for entity in entities])
        entity = entities[0]["entity"]
        new_entity = {"word": word, "entity": entity, "start": entities[0]["start"],
            "end": entities[-1]["end"]}
        return new_entity

    def aggregate_words(self, entities: List[dict]) -> List[dict]:
        """Override tokens from a given word that disagree to force agreement on word boundaries"""
        word_entities = []
        word_group = None
        for entity in entities:
            if word_group is None:
                word_group = [entity]
            elif entity["is_subword"]:
                word_group.append(entity)
            else:
                word_entities.append(self.aggregate_word(word_group))
                word_group = [entity]
        # add last item
        word_entities.append(self.aggregate_word(word_group))
        return word_entities

    def group_sub_entities(self, entities: List[dict]) -> dict:
        """Group together the adjacent tokens with the same entity predicted"""
        # Get the first entity in the entity group
        entity = entities[0]["entity"].split("-")[-1]
        tokens = [entity["word"] for entity in entities]
        entity_group = {"entity_group": entity, "word": self.tokenizer.convert_tokens_to_string(tokens),
            "start": entities[0]["start"], "end": entities[-1]["end"]}
        return entity_group

    def get_tag(self, entity_name: str) -> Tuple[str, str]:
        if entity_name.startswith("B-"):
            bi = "B"
            tag = entity_name[2:]
        elif entity_name.startswith("I-"):
            bi = "I"
            tag = entity_name[2:]
        else:
            # if not in B-, I- format default to I- for continuation
            bi = "I"
            tag = entity_name
        return bi, tag

    def group_entities(self, entities: List[dict]) -> List[dict]:
        """Find and group together the adjacent tokens with the same entity predicted"""
        entity_groups = []
        entity_group_disagg = []
        for entity in entities:
            if not entity_group_disagg:
                entity_group_disagg.append(entity)
                continue
            # if the current entity is similar and adjacent to the previous entity, 
            # append it to the disaggregated entity group
            bi, tag = self.get_tag(entity["entity"])
            last_bi, last_tag = self.get_tag(entity_group_disagg[-1]["entity"])
            if tag == last_tag and bi != "B":
                # modify subword type to be previous_type
                entity_group_disagg.append(entity)
            else:
                # if the current entity is different from the previous entity
                # aggregate the disaggregated entity group
                entity_groups.append(self.group_sub_entities(entity_group_disagg))
                entity_group_disagg = [entity]
        if entity_group_disagg:
            # it's the last entity, add it to the entity groups
            entity_groups.append(self.group_sub_entities(entity_group_disagg))
        return entity_groups

    def build_final_output(self, sentence: str, entity_groups: List[dict]) -> List[dict]:
        entities = []
        for entity_group in entity_groups:
            if entity_group['entity_group'] == 'O':
                continue
            else:
                entry = {}
                entry['start'] = entity_group['start']
                entry['end'] = entity_group['end']
                entry['label'] = entity_group['entity_group']
                entities.append(entry)
        render_data = {'text': sentence, 'ents': entities, 'title': None}
        return render_data
    
if __name__ == '__main__':
    # model_checkpoint = "ner/model/model.onnx"
    sentence = "Jack Sparrow lives in America."
    # sentence = "Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879. Six weeks later the family moved to Munich, where he later on began his schooling at the Luitpold Gymnasium. Later, they moved to Italy and Albert continued his education at Aarau, Switzerland and in 1896 he entered the Swiss Federal Polytechnic School in Zurich to be trained as a teacher in physics and mathematics."
    pipe = NEROnnxModel()
    results = pipe(sentence)
    print(results)

{'input_ids': array([[    1,  2722, 36671,  1074,    11,   730,     4,     2]]), 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1]])}
[[7 7 7 7 7 7 7 7]]
{'sentence': 'Jack Sparrow lives in America.', 'input_ids': array([    1,  2722, 36671,  1074,    11,   730,     4,     2]), 'predictions': array([7, 7, 7, 7, 7, 7, 7, 7]), 'offset_mapping': array([[ 0,  0],
       [ 0,  4],
       [ 4, 12],
       [12, 18],
       [18, 21],
       [21, 29],
       [29, 30],
       [ 0,  0]]), 'special_tokens_mask': array([1, 0, 0, 0, 0, 0, 0, 1])}
{'text': 'Jack Sparrow lives in America.', 'ents': [], 'title': None}


In [107]:
from onnxruntime.quantization import quantize_dynamic, QuantType

model_input = "quant/model.onnx"
model_output = "quant/model-quant1.onnx"
quantize_dynamic(model_input, model_output)

Ignore MatMul due to non constant B: /[MatMul_103]
Ignore MatMul due to non constant B: /[MatMul_134]
Ignore MatMul due to non constant B: /[MatMul_208]
Ignore MatMul due to non constant B: /[MatMul_256]
Ignore MatMul due to non constant B: /[MatMul_356]
Ignore MatMul due to non constant B: /[MatMul_387]
Ignore MatMul due to non constant B: /[MatMul_461]
Ignore MatMul due to non constant B: /[MatMul_509]
Ignore MatMul due to non constant B: /[MatMul_609]
Ignore MatMul due to non constant B: /[MatMul_640]
Ignore MatMul due to non constant B: /[MatMul_714]
Ignore MatMul due to non constant B: /[MatMul_762]
Ignore MatMul due to non constant B: /[MatMul_862]
Ignore MatMul due to non constant B: /[MatMul_893]
Ignore MatMul due to non constant B: /[MatMul_967]
Ignore MatMul due to non constant B: /[MatMul_1015]
Ignore MatMul due to non constant B: /[MatMul_1115]
Ignore MatMul due to non constant B: /[MatMul_1146]
Ignore MatMul due to non constant B: /[MatMul_1220]
Ignore MatMul due to non co